In [1]:
# https://datahacker.rs/019-siamese-network-in-pytorch-with-application-to-face-similarity
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn 
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import os
import random

In [2]:
RECORD_COUNT = 300 # 82736
BATCH_SIZE = 8
TRAIN_START = 0
TRAIN_END = 199
TEST_START = 200
TEST_END = 299
PAIR_COUNT = 80

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [ ]:
device.

In [4]:
xray_df = pd.read_csv(os.path.join("Data_Entry_2017.csv")) 
xray_df = xray_df.head(RECORD_COUNT)

In [5]:
class XRayNetworkDataset(torch.utils.data.Dataset):

    def __init__(self, dataframe, root_dir, start_index, end_index, transform=None, pair_count = PAIR_COUNT):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.start_index = start_index
        self.end_index = end_index
        self.patient_map = {} 
        self.pair_count = pair_count
        self.patient_keys = []

        i = start_index
        while i <= end_index:
            patient_id = self.dataframe.iloc[i]['Patient ID']
            image_path = os.path.join(self.root_dir, dataframe.iloc[i]['Image Index'])
            if patient_id not in self.patient_map:
                self.patient_map[patient_id] = [image_path]
                self.patient_keys.append(patient_id)
            else:
                self.patient_map[patient_id].append(image_path)
            i += 1

    def __len__(self):
        return self.pair_count

    def __getitem__(self, idx):
        
        im0, im1 = None, None
        label = 0

        if (random.randint(0, 1) == 0):
            # Same patient 
            chosen_patient = random.choice(self.patient_keys)
            while len(self.patient_map[chosen_patient]) <= 1:
                chosen_patient = random.choice(self.patient_keys)
            im0, im1 = random.choices(self.patient_map[chosen_patient], k = 2)
            while im1 == im0:
                im0, im1 = random.choices(self.patient_map[chosen_patient], k = 2)
            label = torch.tensor(1)
            
        else:
            # Different patient
            p1, p2 = random.choices(self.patient_keys, k = 2)
            while p1 == p2:
                p1, p2 = random.choices(self.patient_keys, k = 2)
            im0 = random.choice(self.patient_map[p1])
            im1 = random.choice(self.patient_map[p2])
            label = torch.tensor(0)

        im0 = read_image(im0, mode=ImageReadMode.RGB)
        im1 = read_image(im1, mode=ImageReadMode.RGB)

        if self.transform:
            im0 = self.transform(im0)
            im1 = self.transform(im1)
        
        return (im0, im1, label)

In [6]:
dataset = XRayNetworkDataset(xray_df, os.path.join("images"), TRAIN_START, TRAIN_END, transform=transforms.Compose([
    transforms.ConvertImageDtype(torch.float)
]))

In [7]:
train_dataloder = DataLoader(dataset, batch_size=16, num_workers=0)

In [8]:
class SiameseNN (nn.Module):
    def __init__(self): 
        super().__init__()
        self.resnet = models.resnet50(pretrained = True)
        self.resnet.fc = nn.Linear(in_features = self.resnet.fc.in_features, out_features = 128) 
        self.resnet.eval()
        self.fc1 = nn.Linear(in_features = 128, out_features = 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward_once (self, x):
        x = self.resnet(x) 
        return x
    
    def forward(self, x1, x2):
        x1 = self.forward_once(x1)
        x2 = self.forward_once(x2)
        ret = torch.abs(x1 - x2)
        ret = self.fc1(ret)
        ret = self.sigmoid(ret)
        return ret


In [9]:
net = SiameseNN()
net.to(device)

SiameseNN(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
         

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [16]:
for epoch in range(8):

    running_loss = 0.0 

    print (f"Epoch {epoch}")
    for i, data in enumerate(train_dataloder, 0):
        print(data)
        x1, x2, y = data[0].to(device), data[1].to(device), data[2].to(device)
        optimizer.zero_grad()
        fx = net(x1, x2)
        loss = criterion(fx, y) 
        loss.backward()
        optimizer.step()
        current_loss = loss.items()
        running_loss = running_loss + current_loss
        print(f"Epoch {epoch}, batch {i}, loss = {current_loss}") 

    print(f"Epoch {epoch}, Total loss = {running_loss}")

Epoch 0
[tensor([[[[0.0000, 0.5137, 0.8196,  ..., 0.0431, 0.0392, 0.0471],
          [0.0000, 0.5176, 0.8314,  ..., 0.0275, 0.0275, 0.0392],
          [0.0000, 0.5137, 0.8392,  ..., 0.0314, 0.0275, 0.0431],
          ...,
          [0.0000, 0.4471, 0.6510,  ..., 0.0235, 0.0275, 0.0275],
          [0.0000, 0.3569, 0.5569,  ..., 0.0196, 0.0235, 0.0235],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.5137, 0.8196,  ..., 0.0431, 0.0392, 0.0471],
          [0.0000, 0.5176, 0.8314,  ..., 0.0275, 0.0275, 0.0392],
          [0.0000, 0.5137, 0.8392,  ..., 0.0314, 0.0275, 0.0431],
          ...,
          [0.0000, 0.4471, 0.6510,  ..., 0.0235, 0.0275, 0.0275],
          [0.0000, 0.3569, 0.5569,  ..., 0.0196, 0.0235, 0.0235],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.5137, 0.8196,  ..., 0.0431, 0.0392, 0.0471],
          [0.0000, 0.5176, 0.8314,  ..., 0.0275, 0.0275, 0.0392],
          [0.0000, 0.5137, 0.8392

RuntimeError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 4.00 GiB total capacity; 2.46 GiB already allocated; 0 bytes free; 2.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF